In [1]:
import face_recognition
import pickle
import numpy as np
import cv2

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

with open('registered_faces.dat', 'rb') as f1:
	reg_face_encodings = pickle.load(f1)

registered_face_names = list(reg_face_encodings.keys())
registered_face_encodings = np.array(list(reg_face_encodings.values()))

with open('stranger_faces.dat', 'rb') as f2:
	str_face_encodings = pickle.load(f2)

stranger_face_encodings = list(str_face_encodings)

f2.close()

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    pressed_key = cv2.waitKey(1) & 0xFF
    if pressed_key == ord('r'):   #Hit r on the keyboard to recognise the face
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names1 = []
            face_names2 = []
            face_names3 = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches1 = face_recognition.compare_faces(registered_face_encodings, face_encoding)
                matches2 = face_recognition.compare_faces(stranger_face_encodings, face_encoding)
                name1 = "Unknown"
                name2 = "Defaulter"
                name3 = "New User"

                # # If a match was found in known_face_encodings, just use the first one.
                # if True in matches:
                #     first_match_index = matches.index(True)
                #     name = known_face_names[first_match_index]

                # Or instead, use the known face with the smallest distance to the new face
                face_distances1 = face_recognition.face_distance(registered_face_encodings, face_encoding)
                best_match_index1 = np.argmin(face_distances1)
                face_distances2 = face_recognition.face_distance(stranger_face_encodings, face_encoding)
                best_match_index2 = np.argmin(face_distances2)



                if matches1[best_match_index1]:
                    name1 = registered_face_names[best_match_index1]
                    print("Welcome! You are a Registered User")
                    face_names1.append(name1)
                    for (top, right, bottom, left), name1 in zip(face_locations, face_names1):
                        top *= 4
                        right *= 4
                        bottom *= 4
                        left *= 4
                        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                        font = cv2.FONT_HERSHEY_DUPLEX
                        cv2.putText(frame, name1, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                elif matches2[best_match_index2]:
                    print("Sorry! Please go for registration.")
                    face_names2.append(name2)
                    for (top, right, bottom, left), name2 in zip(face_locations, face_names2):
                        top *= 4
                        right *= 4
                        bottom *= 4
                        left *= 4
                        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                        font = cv2.FONT_HERSHEY_DUPLEX
                        cv2.putText(frame, name2, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                else:
                    print("Welcome! You are a new user.")
                    face_names3.append(name3)
                    for (top, right, bottom, left), name1 in zip(face_locations, face_names3):
                        top *= 4
                        right *= 4
                        bottom *= 4
                        left *= 4
                        cv2.rectangle(frame*2, (left, top), (right, bottom), (0, 0, 255), 2)
                        cv2.rectangle(frame*2, (left, bottom), (right, bottom), (0, 0, 255), cv2.FILLED)
                        font = cv2.FONT_HERSHEY_DUPLEX
                        cv2.putText(frame*2, name3, (left + 6, bottom + 6), font, 1.0, (255, 255, 255), 1) 
                        filename="Stranger/stranger_" + str(right) + ".jpg"
                        cv2.imwrite("Stranger/stranger_" + str(right) + ".jpg",img=frame[top:bottom, left:right])
                        print(filename)
                        stranger_face_encodings.append(face_recognition.face_encodings(face_recognition.load_image_file(filename))[0])
                        str_face_encodings = stranger_face_encodings
                        with open('stranger_faces.dat', 'wb') as f3:
                            pickle.dump(str_face_encodings, f3)


            
               

    


   
    

    #Display the resulting image
    

    #Hit 'q' on the keyboard to quit!
    if pressed_key == ord('q'):
        break
        
    else:
        cv2.imshow('Video', frame)

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()



Sorry! Please go for registration.
Sorry! Please go for registration.
